In [1]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##1) Installations and imports


a. Mount drive (if you are running on colab)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


b. Clone or update competition repository
After cloning, under MyDrive, you will see NLI4CT-semeval-2023 folder with the training and dev set, aswell as the full list of CTRs.

In [3]:
%cd /content/drive/MyDrive

import os

PROJECT_DIR = '/content/drive/MyDrive/NLI4CT-semeval-2023'
PROJECT_GITHUB_URL = 'https://github.com/ai-systems/nli4ct.git'

if not os.path.isdir(PROJECT_DIR):
  !git clone {PROJECT_GITHUB_URL}
else:
  %cd {PROJECT_DIR}
  !git pull {PROJECT_GITHUB_URL}

/content/drive/MyDrive
fatal: destination path 'nli4ct' already exists and is not an empty directory.


##2) Dataset

In [4]:
# Training data
!unzip /content/drive/MyDrive/nli4ct/training_data.zip

Archive:  /content/drive/MyDrive/nli4ct/training_data.zip
replace training_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
# Dev set
import json

dev_path = "/content/drive/MyDrive/training_data/dev.json"
with open(dev_path) as json_file:
    dev = json.load(json_file)

# Example instance
print(dev[list(dev.keys())[1]])

{'Type': 'Comparison', 'Section_id': 'Eligibility', 'Primary_id': 'NCT00425854', 'Secondary_id': 'NCT01224678', 'Statement': 'Patients with significantly elevated ejection fraction are excluded from the primary trial, but can still be eligible for the secondary trial if they are 55 years of age or over', 'Label': 'Contradiction', 'Primary_evidence_index': [15], 'Secondary_evidence_index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}


In [6]:
uuid_list = list(dev.keys())
statements = []
gold_dev_primary_evidence = []
gold_dev_secondary_evidence = []
for i in range(len(uuid_list)):
  #Retrieve all statements from the development set
  statements.append(dev[uuid_list[i]]["Statement"])

##3) BM25 Evidence Retrieval baseline

In [7]:
Results = {}

for i in range(len(uuid_list)):

  primary_ctr_path = os.path.join("/content/drive/MyDrive/training_data/CT json",dev[uuid_list[i]]["Primary_id"]+".json")
  with open(primary_ctr_path) as json_file:
    primary_ctr = json.load(json_file)
  primary_section = primary_ctr[dev[uuid_list[i]]["Section_id"]]

  # Tokenize the section entries, remove spaces, and empty entries
  tokenized_primary = [x.split(" ") for x in primary_section]
  tokenized_primary = [[x.strip(' ') for x in y] for y in tokenized_primary]
  tokenized_primary = [[x for x in y if x] for y in tokenized_primary]

  #create an instance of the BM25 class, which reads in the primary section text and does some indexing on it
  bm25 = BM25Okapi(tokenized_primary)

  #retrieve and tokenize the statement
  statement = statements[i].split(" ")

  #Retrieve bm25 scores for the primary section
  primary_scores = bm25.get_scores(statement)

  #Retrieve all entries from the primary section with a bm25 score over 1
  primary_retrieved = []
  for j in range(len(primary_section)):
    if primary_scores[j]>1:
      primary_retrieved.append(j)

  #Repeat for the secondary trial
  if dev[uuid_list[i]]["Type"] == "Comparison":
    secondary_ctr_path = os.path.join("/content/drive/MyDrive/training_data/CT json",dev[uuid_list[i]]["Secondary_id"]+".json")
    with open(secondary_ctr_path) as json_file:
      secondary_ctr = json.load(json_file)
    secondary_section = secondary_ctr[dev[uuid_list[i]]["Section_id"]]

    # Tokenize the section entries, remove spaces, and empty entries
    tokenized_secondary = [x.split(" ") for x in secondary_section]
    tokenized_secondary = [[x.strip(' ') for x in y] for y in tokenized_secondary]
    tokenized_secondary = [[x for x in y if x] for y in tokenized_secondary]

    #create an instance of the BM25 class, which reads in the secondary section text and does some indexing on it
    bm25 = BM25Okapi(tokenized_secondary)

    #Retrieve bm25 scores for the secondary section
    secondary_scores = bm25.get_scores(statement)

    #Retrieve all entries from the secondary section with a bm25 score over 1
    secondary_retrieved = []
    for j in range(len(secondary_section)):
      if secondary_scores[j]>1:
        secondary_retrieved.append(j)
    Results[str(uuid_list[i])] = {"Primary_evidence_index":primary_retrieved,"Secondary_evidence_index":secondary_retrieved}
  else:
    Results[str(uuid_list[i])] = {"Primary_evidence_index":primary_retrieved}


## Save the results in the submission format.

In [8]:
print(Results)
with open("/content/drive/MyDrive/nli4ct/results.json",'w') as jsonFile:
    jsonFile.write(json.dumps(Results,indent=4))

{'1adc970c-d433-44d0-aa09-d3834986f7a2': {'Primary_evidence_index': [2]}, '6b9162d0-0816-46d4-81af-c60028dcc63b': {'Primary_evidence_index': [2, 3, 6, 7, 12, 13, 15, 18, 20, 22, 23, 25], 'Secondary_evidence_index': [0, 1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15]}, '0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f': {'Primary_evidence_index': [5, 6, 18, 19], 'Secondary_evidence_index': []}, 'cc1f712a-2116-4e40-9810-f315e3fa5ff8': {'Primary_evidence_index': [6, 10]}, '904061c0-14fa-4f13-9118-9a41e24fa8eb': {'Primary_evidence_index': [1, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17, 23]}, '43ee7645-ce1e-42d5-9a74-3e379f6f367b': {'Primary_evidence_index': [3, 4, 5, 8, 10, 12, 13, 16, 21, 29, 30, 33]}, '0cef8c8e-7986-46c7-a597-c5733a9899c0': {'Primary_evidence_index': [2, 3, 9, 15]}, '43ce26e5-03fa-4e9d-b0eb-6ea356295753': {'Primary_evidence_index': [6]}, '3facad41-0221-42f8-834d-470e65c4aad5': {'Primary_evidence_index': [1, 2, 7]}, '9cbc00e9-3a2d-4471-a93e-72c95132fb6a': {'Primary_evidence_index': [2, 5

##4) Evaluation

Run the task 2 evaluation script.

In [9]:
def main():

    gold = dev
    results = Results  
    uuid_list = list(results.keys())

    results_p = []
    gold_p =[]
    results_s = []
    gold_s =[]

    for i in range(len(uuid_list)):
        gold_p.append(gold[uuid_list[i]]["Primary_evidence_index"])
        results_p.append(results[uuid_list[i]]["Primary_evidence_index"])
        if gold[uuid_list[i]]["Type"]=="Comparison":
            gold_s.append(gold[uuid_list[i]]["Secondary_evidence_index"])
            results_s.append(results[uuid_list[i]]["Secondary_evidence_index"])


    TP = 0
    FP = 0
    FN = 0

    for i in range(len(gold_p)):
        for j in range(len(results_p[i])):
            if results_p[i][j] in gold_p[i]:
                TP += 1
            if results_p[i][j] not in gold_p[i]:
                FP += 1
        for j in range(len(gold_p[i])):
            if gold_p[i][j] not in results_p[i]:
                FN += 1

    for i in range(len(gold_s)):
        for j in range(len(results_s[i])):
            if results_s[i][j] in gold_s[i]:
                TP += 1
            if results_s[i][j] not in gold_s[i]:
                FP += 1
        for j in range(len(gold_s[i])):
            if gold_s[i][j] not in results_s[i]:
                FN += 1

    p_score = TP / (TP + FP)
    r_score = TP / (TP + FN)
    score = 2*(p_score * r_score)/(p_score + r_score)

    print('F1:{:f}'.format(score))
    print('precision_score:{:f}'.format(p_score))
    print('recall_score:{:f}'.format(r_score))
        
if '__main__' == __name__:
    main()



F1:0.322748
precision_score:0.422034
recall_score:0.261280
